In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def drop_by_index(X,indexes):
    """
    helper function to drop rows of dataframe and return new dataframe without those rows with indexes resetted
    """
    X = X.drop(indexes)
    X = X.reset_index().drop(columns="index")
    return(X)

def calculate_death_ratio(data):
    """
    Given data, calculate survivability ratios for DFS, OS, CSS
    input: dataframe of patients. this data should be a pre-filtered list of patients
    output: None as of this version. (WIP)
    """
    counter_5y = 0
    counter_10y = 0
    counter_DFS = [0,0]
    counter_CSS = [0,0]
    counter_OS = [0,0]
    for index, row in data.iterrows():
#         print(row)
        CSS = row["CSS_years"]
        DFS = row["DFS_years"]
        OS = row["OS_years"]

        # if patient is alive
        if np.isnan(row["death_age"]):
            max_years = max(CSS,DFS,OS)
            if max_years >= 10:
                counter_5y += 1
                counter_10y += 1

                if DFS >= 10:
                    counter_DFS[0] += 1
                    counter_DFS[1] += 1
                elif DFS >= 5:
                    counter_DFS[0] += 1

                if OS >= 10:
                    counter_OS[0] += 1
                    counter_OS[1] += 1
                elif DFS >= 5:
                    counter_OS[0] += 1

                if CSS >= 10:
                    counter_CSS[0] += 1
                    counter_CSS[1] += 1
                elif DFS >= 5:
                    counter_CSS[0] += 1
                    
            elif max_years >= 5:
                counter_5y += 1

                if DFS >= 5:
                    counter_DFS[0] += 1

                if DFS >= 5:
                    counter_OS[0] += 1

                if DFS >= 5:
                    counter_CSS[0] += 1
                    
        # if patient is dead
        else:
            counter_5y += 1
            counter_10y += 1
            
            if DFS >= 10:
                counter_DFS[0] += 1
                counter_DFS[1] += 1
            elif DFS >= 5:
                counter_DFS[0] += 1
                
            if OS >= 10:
                counter_OS[0] += 1
                counter_OS[1] += 1
            elif DFS >= 5:
                counter_OS[0] += 1
                
            if CSS >= 10:
                counter_CSS[0] += 1
                counter_CSS[1] += 1
            elif DFS >= 5:
                counter_CSS[0] += 1
    print("Across the data, survival ratios are as follows")
    print("DFS\n5 Year: {:.2f}%\n10 Year: {:.2f}%".format(counter_DFS[0]/counter_5y*100,counter_DFS[1]/counter_10y*100))
    print("OS\n5 Year: {:.2f}%\n10 Year: {:.2f}%".format(counter_OS[0]/counter_5y*100,counter_OS[1]/counter_10y*100))
    print("CSS\n5 Year: {:.2f}%\n10 Year: {:.2f}%".format(counter_CSS[0]/counter_5y*100,counter_CSS[1]/counter_10y*100))
                
                

In [ ]:
clinical = pd.read_pickle("C:\\SMU_v2\\clinical_output.pkl").reset_index().drop(columns="index")
to_drop = clinical[clinical['dx_date']=="NA"].index
clinical = drop_by_index(clinical,to_drop)

In [ ]:
survival = clinical[["NRIC","dx_date","Date_for_DFS","Date_for_OS","Date_for_CSS","death_age"]]

In [ ]:
survival = survival.drop(survival[survival['Date_for_DFS'].isnull()|survival['Date_for_OS'].isnull()|survival['Date_for_CSS'].isnull()].index)

In [ ]:
survival["Date_for_DFS"] = pd.to_datetime(survival["Date_for_DFS"])
survival["Date_for_OS"] = pd.to_datetime(survival["Date_for_OS"])
survival["Date_for_CSS"] = pd.to_datetime(survival["Date_for_CSS"])
survival["dx_date"] = pd.to_datetime(survival["dx_date"])


In [ ]:
survival["DFS_years"] = (survival["Date_for_DFS"] - survival['dx_date'] )/np.timedelta64(1, 'Y')//1
survival["OS_years"] = (survival["Date_for_OS"] - survival['dx_date'] )/np.timedelta64(1, 'Y')//1
survival["CSS_years"] = (survival["Date_for_CSS"] - survival['dx_date'] )/np.timedelta64(1, 'Y')//1

In [ ]:
calculate_death_ratio(survival)